In [ ]:
import seaborn as sns
sns.set()

In [ ]:
from static_grader import grader

# The New York Social Graph


[New York Social Diary](https://web.archive.org/web/20150913224145/http://www.newyorksocialdiary.com/) provides a
fascinating lens onto New York's socially well-to-do.  The data forms a natural social graph for New York's social elite.  Take a look at this page of a recent [run-of-the-mill holiday party](https://web.archive.org/web/20150913224145/http://www.newyorksocialdiary.com/party-pictures/2014/holiday-dinners-and-doers). Please note that these links point to the internet archive, as the original website has recently removed most of its archives. Many of the images no longer load, but all the HTML is still there.

Besides the brand-name celebrities, you will notice the photos have carefully annotated captions labeling those that appear in the photos.  We can think of this as implicitly implying a social graph: there is a connection between two individuals if they appear in a picture together.

For this project, we will assemble the social graph from photo captions for parties dated December 1, 2014, and before.  Using this graph, we can make guesses at the most popular socialites, the most influential people, and the most tightly coupled pairs.

We will attack the project in three phases:
1. Get a list of all the photo pages to be analyzed.
2. Parse all of the captions on a sample page.
3. Parse all of the captions on all pages, and assemble the graph.

## Phase One


The first step is to crawl the data.  We want photos from parties on or before December 1st, 2014.  Go to the [Party Pictures Archive](https://web.archive.org/web/20150913224145/http://www.newyorksocialdiary.com/party-pictures) to see a list of (party) pages.  We want to get the url for each party page, along with its date.

Here are some packages that you may find useful.  You are welcome to use others, if you prefer.

In [ ]:
import requests
import dill
from bs4 import BeautifulSoup
from datetime import datetime
import re
import numpy as np
import os 
os.environ['KMP_DUPLICATE_LIB_OK']='True'

We recommend using Python [Requests](http://docs.python-requests.org/en/master/) to download the HTML pages, and [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/) to process the HTML.  Let's start by getting the [first page](https://web.archive.org/web/20150913224145/http://www.newyorksocialdiary.com/party-pictures).

In [ ]:
url = "https://web.archive.org/web/20150913224145/http://www.newyorksocialdiary.com/party-pictures"
page = requests.get(url) # Use requests.get to download the page.


Now, we process the text of the page with BeautifulSoup.

In [ ]:
soup = BeautifulSoup(page.text, "lxml") # lxml gets rid of the parser choice warning

This page has links to 50 party pages. Look at the structure of the page and determine how to isolate those links.  Your browser's developer tools (usually `Cmd`-`Option`-`I` on Mac, `Ctrl`-`Shift`-`I` on others) offer helpful tools to explore the structure of the HTML page.

Once you have found a pattern, use BeautifulSoup's [select](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#css-selectors) or [find_all](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#find) methods to get those elements.

In [ ]:
links =soup.find_all('div', attrs={'class': 'views-row'})

There should be 50 per page.

In [ ]:
assert len(links) == 50

Let's take a look at that first link.  Figure out how to extract the URL of the link, as well as the date.  You probably want to use `datetime.strptime`.  See the [format codes for dates](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior) for reference.

In [ ]:
link = links[0]
# Check that the title and date match what you see visually.

For purposes of code reuse, let's put that logic into a function.  It should take the link element and return the URL and date parsed from it.

In [ ]:
def get_link_date(el):
   #get the link
    url = 'https://web.archive.org/web/20150913224145/http://www.newyorksocialdiary.com' + el.a['href']   
    #get the date   
    #first child span
    first_span = el.find('span', attrs={'class': 'views-field-created'})
    #seond child spand that is actually the date of the party
    second_span = first_span.find('span', attrs = {'class' : 'field-content'}).text.encode('utf-8')
    #this following party_time retunrs the dates as strings
    #party_date = datetime.strptime(second_span, '%A, %B %d, %Y').strftime('%Y, %m, %d')
    #following returns the dates as date times
    date = datetime.strptime(second_span.decode('ascii'), '%A, %B %d, %Y')
    return url, date

You may want to check that it works as you expected.

Once that's working, let's write another function to parse all of the links on a page.  Thinking ahead, we can make it take a Requests [Response](https://requests.readthedocs.io/en/master/api/#requests.Response) object and do the BeautifulSoup parsing within it.

In [ ]:
def get_links(response):
    response.text[:1000] + "..."
    soup = BeautifulSoup(response.text, "lxml")
    l = soup.find_all('div', attrs={'class':'views-row'})
    links = []
    for link in l:
        links.append(get_link_date(link))
    return links # A list of URL, date pairs

If we run this on the previous response, we should get 50 pairs.

In [ ]:
get_links(page)

But we only want parties with dates on or before the first of December, 2014.  Let's write a function to filter our list of dates to those at or before a cutoff.  Using a keyword argument, we can put in a default cutoff, but allow us to test with others.

In [ ]:
def filter_by_date(links, cutoff=datetime(2014, 12, 1)):
    before2014 = []
    for l in links:
        if l[1] <= cutoff:
            before2014.append(l)
    return before2014
    # Return only the elements with date <= cutoff

With the default cutoff, there should be no valid parties on the first page.  Adjust the cutoff date to check that it is actually working.

In [ ]:
assert len(filter_by_date(get_links(page))) == 0

Now we should be ready to get all of the party URLs.  Click through a few of the index pages to determine how the URL changes.  Figure out a strategy to visit all of them.

HTTP requests are generally IO-bound.  This means that most of the time is spent waiting for the remote server to respond.  If you use `requests` directly, you can only wait on one response at a time.  [requests-futures](https://github.com/ross/requests-futures) lets you wait for multiple requests at a time.  You may wish to use this to speed up the downloading process.

In [ ]:
from requests_futures.sessions import FuturesSession

link_list = []
session = FuturesSession(max_workers=5)
urls = ["https://web.archive.org/web/20150913224145/http://www.newyorksocialdiary.com/party-pictures?page=" + str(i) for i in range(26)]
futures = [session.get(url) for url in urls]
for future in futures:
    link_list.extend(get_links(future.result()))
    
link_list = filter_by_date(link_list, cutoff=datetime(2014, 12, 1))
# You can use link_list.extend(others) to add the elements of others
# to link_list.

In the end, you should have 1193 parties.

In [ ]:
assert len(link_list) == 1193

In case we need to restart the notebook, we should save this information to a file.  There are many ways you could do this; here's one using `dill`.

In [ ]:
dill.dump(link_list, open('nysd-links.pkd', 'wb'))

To restore the list, we can just load it from the file.  When the notebook is restarted, you can skip the code above and just run this command.

In [ ]:
link_list = dill.load(open('nysd-links.pkd', 'rb'))

## Question 1: histogram


Get the number of party pages for the 95 months (that is, month-year pair) in the data.  Represent this histogram as a list of 95 tuples, each of the form `("Dec-2014", 1)`.  Note that you can convert `datetime` objects into these sort of strings with `strftime` and the [format codes for dates](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior).

Plot the histogram for yourself.  Do you see any trends?

In [ ]:
date_of_parties = [l[1] for l in link_list]
month_of_parties = [d.strftime('%b-%Y') for d in date_of_parties]

link_list_months = []

for i in range(len(link_list)):
    link_list_months.append([link_list[i][0], month_of_parties[i]])

In [ ]:
from collections import Counter
def histogram():   
     return list(Counter(month_of_parties).items())

In [ ]:
grader.score('graph__histogram', histogram)

## Phase Two


In this phase, we concentrate on getting the names out of captions for a given page.  We'll start with [the benefit cocktails and dinner](https://web.archive.org/web/20150913224145/http://www.newyorksocialdiary.com/party-pictures/2015/celebrating-the-neighborhood) for [Lenox Hill Neighborhood House](http://www.lenoxhill.org/), a neighborhood organization for the East Side.

Take a look at that page.  Note that some of the text on the page is captions, but others are descriptions of the event.  Determine how to select only the captions.

In [ ]:
captions = ...

By our count, there are about 110.  But if you're off by a couple, you're probably okay.

In [ ]:
assert abs(len(captions) - 110) < 5

Let's encapsulate this in a function.  As with the links pages, we want to avoid downloading a given page the next time we need to run the notebook.  While we could save the files by hand, as we did before, a checkpoint library like [ediblepickle](https://pypi.python.org/pypi/ediblepickle/1.1.3) can handle this for you.  (Note, though, that you may not want to enable this until you are sure that your function is working.)

You should also keep in mind that HTTP requests fail occasionally, for transient reasons.  You should plan how to detect and react to these failures.   The [retrying module](https://pypi.python.org/pypi/retrying) is one way to deal with this.

In [ ]:
def get_captions(path):
    response = requests.get(path)
    soup = BeautifulSoup(response.text, "lxml")
    caption_divs_main = soup.find_all('font', attrs={'face': 'Verdana, Arial, Helvetica, sans-serif'})
    caption_divs2 = soup.find_all("div", {'align': 'center', 'class': 'photocaption'}, text=True)
    caption_divs3 = soup.find_all("td", {'valign': 'top', 'class': 'photocaption'}, text=True)
    caption_divs4 = soup.find_all("td", {'valign': 'top', 'class': 'photocaption', 'style': 'background-color:#faf9ee'}, text=True)
    caps = caption_divs_main + caption_divs2 + caption_divs3 + caption_divs4 
    list_of_captions = []
    for tag in caps:
        list_of_captions.append(tag.text.strip().encode('utf-8'))
    return list_of_captions

This should get the same captions as before.

In [ ]:
captions_sample = get_captions("https://web.archive.org/web/20151114014941/http://www.newyorksocialdiary.com/party-pictures/2015/celebrating-the-neighborhood")
dill.dump(captions_sample, open('captions_sample.pkd', 'wb'))

Now that we have some sample captions, let's start parsing names out of those captions.  There are many ways of going about this, and we leave the details up to you.  Some issues to consider:

  1. Some captions are not useful: they contain long narrative texts that explain the event.  Try to find some heuristic rules to separate captions that are a list of names from those that are not.  A few heuristics include:
    - look for sentences (which have verbs) and as opposed to lists of nouns. For example, [`nltk` does part of speech tagging](http://www.nltk.org/book/ch05.html) but it is a little slow. There may also be heuristics that accomplish the same thing.
    - Similarly, spaCy's [entity recognition](https://spacy.io/docs/usage/entity-recognition) could be useful here.
    - Look for commonly repeated threads (e.g. you might end up picking up the photo credits or people such as "a friend").
    - Long captions are often not lists of people.  The cutoff is subjective, but for grading purposes, *set that cutoff at 250 characters*.
  2. You will want to separate the captions based on various forms of punctuation.  Try using `re.split`, which is more sophisticated than `string.split`. **Note**: The reference solution uses regex exclusively for name parsing.
  3. You might find a person named "ra Lebenthal".  There is no one by this name.  Can anyone spot what's happening here?
  4. This site is pretty formal and likes to say things like "Mayor Michael Bloomberg" after his election but "Michael Bloomberg" before his election.  Can you find other ('optional') titles that are being used?  They should probably be filtered out because they ultimately refer to the same person: "Michael Bloomberg."
  5. There is a special case you might find where couples are written as eg. "John and Mary Smith". You will need to write some extra logic to make sure this properly parses to two names: "John Smith" and "Mary Smith".
  6. When parsing names from captions, it can help to look at your output frequently and address the problems that you see coming up, iterating until you have a list that looks reasonable. This is the approach used in the reference solution. Because we can only asymptotically approach perfect identification and entity matching, we have to stop somewhere.
  
**Questions worth considering:**
  1. Who is Patrick McMullan and should he be included in the results? How would you address this?
  2. What else could you do to improve the quality of the graph's information?

In [ ]:
names = [s.decode('utf-8', 'ignore') for s in captions_sample]
names

In [ ]:
names = [s.decode('utf-8', 'ignore') for s in captions_sample]
n = []
for name in names:
    n.append(spacy_parse_string(name))
n

In [ ]:
sample_names = []
for i in range(len(n)):
    for j in n[i]:
        sample_names.append(j)
sample_names

## Question 2: sample_names


Once you feel that your algorithm is working well on these captions, parse all of the captions and extract all the names mentioned.  Sort them alphabetically, by first name, and return the first hundred.

In [ ]:
import numpy as np
sample_n =np.sort(sample_names)
sample_n = np.unique(sample_n)
sample_n = sample_n[0:100]
sample_n

In [ ]:
grader.score('graph__sample_names', sample_n)

Now, run this sort of test on a few other pages.  You will probably find that other pages have a slightly different HTML structure, as well as new captions that trip up your caption parser.  But don't worry if the parser isn't perfect -- just try to get the easy cases.

## Phase Three


Once you are satisfied that your caption scraper and parser are working, run this for all of the pages.  If you haven't implemented some caching of the captions, you probably want to do this first.

In [ ]:
# Scraping all of the pages could take 10 minutes or so.

In [ ]:
all_captions = []
for i in range(len(link_list)):
    loc = link_list[i][0].find('http://www.newyorksocialdiary.com/party-pictures')
    pic_path ='https://web.archive.org/web/20151114014941/'+link_list[i][0][loc:]
    all_captions.extend(get_captions(pic_path))

In [ ]:
len(all_captions)

In [ ]:
dill.dump(all_captions, open('all_cap.pkd', 'wb'))

For the remaining analysis, we think of the problem in terms of a
[network](http://en.wikipedia.org/wiki/Computer_network) or a
[graph](https://en.wikipedia.org/wiki/Graph_%28discrete_mathematics%29).  Any time a pair of people appear in a photo together, that is considered a link.  What we have described is more appropriately called an (undirected)
[multigraph](http://en.wikipedia.org/wiki/Multigraph) with no self-loops but this has an obvious analog in terms of an undirected [weighted graph](http://en.wikipedia.org/wiki/Graph_%28mathematics%29#Weighted_graph).  In this problem, we will analyze the social graph of the New York social elite.  We recommend using python's [`networkx`](https://networkx.github.io/) library.

In [ ]:
import spacy

nlp = spacy.load('en_core_web_sm')

In [ ]:
def spacy_parse_string(party_string):
    doc = nlp(party_string)
    
    list_of_names = [ent.text for ent in doc.ents if ent.label_ =='PERSON']
    for ind in range(len(list_of_names)-2,-1,-1):
        if len(list_of_names[ind].split(" ")) == 1 and len(list_of_names[ind+1].split(" ")) !=1:
            list_of_names[ind] = list_of_names[ind] +  " "+ list_of_names[ind+1].split(" ")[1]
    return list_of_names
            

In [ ]:
all_captions = dill.load(open('all_cap.pkd', 'rb'))
all_names = [s.decode('utf-8', 'ignore') for s in all_captions]
n = []
for name in all_names:
    n.append(spacy_parse_string(name))

In [ ]:
names = []
for i in range(len(n)):
    for j in n[i]:
        names.append(j)

In [ ]:
len(names)

In [ ]:
dill.dump(names, open('names.pkd', 'wb'))

All in all, you should end up with over 100,000 captions and more than 110,000 names, connected in about 200,000 pairs.

## Question 3: degree


The simplest question to ask is "who is the most popular"?  The easiest way to answer this question is to look at how many connections everyone has.  Return the top 100 people and their degree.  Remember that if an edge of the graph has weight 2, it counts for 2 in the degree.

**Checkpoint:** Some aggregate stats on the solution

    "count": 100.0
    "mean": 189.92
    "std": 87.8053034454
    "min": 124.0
    "25%": 138.0
    "50%": 157.0
    "75%": 195.0
    "max": 666.0

In [ ]:
import itertools  # itertools.combinations may be useful
import networkx as nx
from itertools import combinations
import heapq                # Heaps are efficient structures for tracking the largest
              # elements in a collection.  Use introspection to find the
              # function you need.

In [ ]:
G = nx.MultiGraph()
def degree():
    for caption in n:
            connections = list(combinations(caption,2))
            G.add_edges_from(list(combinations(caption,2)))
    A = sorted(G.degree(), key = lambda x: -x[1])[:100]
    return A

In [ ]:
len(degree())

In [ ]:
grader.score('graph__degree', degree)

## Question 4: PageRank


A similar way to determine popularity is to look at their
[PageRank](http://en.wikipedia.org/wiki/PageRank).  PageRank is used for web ranking and was originally
[patented](http://patft.uspto.gov/netacgi/nph-Parser?patentnumber=6285999) by Google and is essentially the stationary distribution of a [Markov
chain](http://en.wikipedia.org/wiki/Markov_chain) implied by the social graph. You can implement this yourself or use the version in `networkx`

Use 0.85 as the damping parameter so that there is a 15% chance of jumping to another vertex at random.

**Checkpoint:** Some aggregate stats on the solution

    "count": 100.0
    "mean": 0.0001841088
    "std": 0.0000758068
    "min": 0.0001238355
    "25%": 0.0001415028
    "50%": 0.0001616183
    "75%": 0.0001972663
    "max": 0.0006085816

In [ ]:
import itertools  # itertools.combinations may be useful
import networkx as nx
from itertools import combinations

def pagerank():
    G = nx.MultiGraph()
    for caption in n:
            connections = list(combinations(caption,2))
            G.add_edges_from(list(combinations(caption,2)))
       
    H = nx.Graph()
    for u,v,w in G.edges(data=True):
        if H.has_edge(u,v):
            H[u][v]['weight'] += 1
        else:
            H.add_edge(u,v,weight=1)
    ranks = nx.pagerank(H)
    A = sorted(ranks.items(), key = lambda x: -x[1])[:100]
    perfect_rank = []   
    for i in range(100):
        perfect_rank.append(A[i])
    return perfect_rank

In [ ]:
grader.score('graph__pagerank', pagerank)

Another interesting question is who tend to co-occur with each other.  Give us the 100 edges with the highest weights.

Google these people and see what their connection is.  Can we use this to detect instances of infidelity?

**Checkpoint:** Some aggregate stats on the solution

    "count": 100.0
    "mean": 25.84
    "std": 16.0395470855
    "min": 14.0
    "25%": 16.0
    "50%": 19.0
    "75%": 29.25
    "max": 109.0

In [ ]:
import pandas as pd

weights = H.edges(data = True)
L = []
for (n1,n2,w) in weights:
    t = (n1,n2,w['weight'])
    L.append(t)
df = pd.DataFrame(L, columns=['node1', 'node2', 'weight'])
sorted_df = df.sort_values(['weight'],ascending = False)
 
best_friends = []
for name1,name2,weight in sorted_df[0:100].values:
    best_friends.append(((str(name1),str(name2)),weight))

In [ ]:
grader.score('graph__best_friends', best_friends)

In [ ]:
CAPTIONS = [
    u"Robin and Chad Pennington",
    u"Professor Saima Wazed Hossain with  H.E. Madame Ilham Hussein, first lady of the Maldives",
    u"Bob and Claire Rigel with Tom Hall",
    u"Count and Countess Dijon von Monteton ",
    u"Mauro Bergonzoli and Franziska Countess Fugger von Babenhausen",
    u"Carmen and Zarin Mehta",
    u"Board Chair Brooke Garber Neidich, Dr. Ruth Westheimer, President Harold S. Koplewicz, MD, Michael Fascitelli, Governor Jon S. Corzine, and Speaker Christine Quinn",
    u" Dr. Mario Lacouture with CancerCare Director of Education and Training Carolyn Messner ",
    u" Bono             and Harry Belafonte",
    u"       Catherine Sabino, Christophe von Hohenberg, and Caroline von  Humboldt.    ",
    u"Bonnie Strauss, President and Founder, with Morris and Nancy Offit"
]

def caption_tester(f):
    for c in CAPTIONS:
        print(c)
        print(f(c))
        print()

*Copyright &copy; 2020 Pragmatic Institute. This content is licensed solely for personal use. Redistribution or publication of this material is strictly prohibited.*